<h3> Data Wrangling </h3>

Full trip data for the years 2010 to 2019, downloaded directly from Capital Bike Share website. <br>

Data types (columns) change from 2020-04 onwards. <br>
For the sake of having complete years in data set, this analysis only includes up to 2019


Download link: https://s3.amazonaws.com/capitalbikeshare-data/index.html







In [66]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import copy
import math
import glob
import os
import seaborn as sns

In [67]:
path = r"/Users/Oh/Documents/CodeAcademyBerlin/bike_analysis/data/years_2010_to_2019"
all_files = glob.glob(os.path.join(path, "*.csv"))
all_files.sort() # Files are arranged already in correct alphabetical order in the folder, so we want to preserve the order.

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

Run time: 58 s

<h3> Dataframe with daily values </h3>
First, create a dataframe for keeping track of daily values. <br>
Further below, will create a df for hourly values.

In [68]:
# Create a dataframe for the daily values

df_all_years = concatenated_df.copy(deep = True)        # Want to preserve the original "concatenated_df"

run time = 37 s

In [69]:
df_all_years.columns

Index(['Duration', 'Start date', 'End date', 'Start station number',
       'Start station', 'End station number', 'End station', 'Bike number',
       'Member type'],
      dtype='object')

In [70]:
df_all_years['Start date'] = pd.to_datetime(df_all_years['Start date'])

In [71]:
df_all_years['End date'] = pd.to_datetime(df_all_years['End date'])

In [72]:
df_all_years.head(2)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,M St & New Jersey Ave SE,31108,4th & M St SW,W00742,Member
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,1st & N St SE,31209,1st & N St SE,W00032,Member


In [73]:
df_all_years.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26058744 entries, 0 to 26058743
Data columns (total 9 columns):
 #   Column                Dtype         
---  ------                -----         
 0   Duration              int64         
 1   Start date            datetime64[ns]
 2   End date              datetime64[ns]
 3   Start station number  int64         
 4   Start station         object        
 5   End station number    int64         
 6   End station           object        
 7   Bike number           object        
 8   Member type           object        
dtypes: datetime64[ns](2), int64(3), object(4)
memory usage: 1.7+ GB


Select rows where Member type == 'Member' (i.e. 'registered') <br>
Then do groupby + agg to find total number of registered users per day

In [74]:
df_agg_registered = df_all_years [df_all_years['Member type'] == 'Member'].groupby(pd.Grouper(key = 'Start date', freq = 'D')).count().rename(columns={'Member type': 'Registered'})
df_agg_registered.reset_index(inplace = True)
df_agg_registered = df_agg_registered[['Start date', 'Registered']]


In [75]:
df_agg_registered.head()

,Start date,Registered
0,2010-09-20,178
1,2010-09-21,215
2,2010-09-22,260
3,2010-09-23,249
4,2010-09-24,206


Select rows where Member type == 'Casual' <br>
Then do groupby + agg to find total number of casual users per day

In [76]:
df_agg_casual = df_all_years [df_all_years['Member type'] == 'Casual'].groupby(pd.Grouper(key = 'Start date', freq = 'D')).count().rename(columns={'Member type': 'Casual'})
df_agg_casual = df_agg_casual.reset_index()
df_agg_casual = df_agg_casual [['Start date', 'Casual']]

In [77]:
df_agg_casual.head()

,Start date,Casual
0,2010-09-20,34
1,2010-09-21,109
2,2010-09-22,117
3,2010-09-23,124
4,2010-09-24,156


In [78]:
# Merge df_agg_registered with df_agg_casual

df_all_agg_daily = pd.merge(df_agg_registered, df_agg_casual, how = 'outer', on = ['Start date'])
df_all_agg_daily = df_all_agg_daily.rename(columns = {'Start date': 'date', 
                                            'Registered': 'registered',
                                            'Casual': 'casual'})

In [79]:
df_all_agg_daily.head()

,date,registered,casual
0,2010-09-20,178,34
1,2010-09-21,215,109
2,2010-09-22,260,117
3,2010-09-23,249,124
4,2010-09-24,206,156


In [80]:
df_all_agg_daily.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3390 entries, 0 to 3389
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        3390 non-null   datetime64[ns]
 1   registered  3390 non-null   int64         
 2   casual      3390 non-null   int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 105.9 KB


In [81]:
df_all_agg_daily.isna().sum()

date          0
registered    0
casual        0
dtype: int64

In [82]:
df_all_agg_daily['year'] = df_all_agg_daily['date'].dt.year
df_all_agg_daily['month'] = df_all_agg_daily['date'].dt.month
df_all_agg_daily['day'] = df_all_agg_daily['date'].dt.day_name()
df_all_agg_daily['day'] = df_all_agg_daily['day'].apply(lambda x: x[0:3])

In [83]:
df_all_agg_daily.head()

,date,registered,casual,year,month,day
0,2010-09-20,178,34,2010,9,Mon
1,2010-09-21,215,109,2010,9,Tue
2,2010-09-22,260,117,2010,9,Wed
3,2010-09-23,249,124,2010,9,Thu
4,2010-09-24,206,156,2010,9,Fri


Now we want to create the same kind of data, but on an hourly basis.

In [84]:
df_all_years_hourly = concatenated_df.copy(deep = True)

In [85]:
df_all_years_hourly.tail(2)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
26058742,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member
26058743,183,2019-12-31 23:59:55,2020-01-01 00:02:58,31227,13th St & New York Ave NW,31254,15th & K St NW,W22782,Member


In [86]:
df_all_years_hourly['Start date'] = pd.to_datetime(df_all_years_hourly['Start date'])
df_all_years_hourly['End date'] = pd.to_datetime(df_all_years_hourly['End date'])
df_all_years_hourly['hour'] = df_all_years_hourly['Start date'].dt.hour

In [87]:
df_all_years_hourly.tail(2)

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type,hour
26058742,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member,23
26058743,183,2019-12-31 23:59:55,2020-01-01 00:02:58,31227,13th St & New York Ave NW,31254,15th & K St NW,W22782,Member,23


Select rows where Member type == 'Member' (i.e. 'registered') <br>
Then do groupby + agg to find total number of registered users per hour

In [88]:
df_agg_registered_hourly = df_all_years_hourly [df_all_years_hourly['Member type'] == 'Member'].groupby(pd.Grouper(key = 'Start date', freq = 'H')).count().rename(columns={'Member type': 'Registered'})

In [89]:
df_agg_registered_hourly = df_agg_registered_hourly[['Registered']]

In [90]:
df_agg_registered_hourly.tail(2)

,Registered
Start date,
2019-12-31 22:00:00,71
2019-12-31 23:00:00,83


In [91]:
# This contains number of registered users for each day.
df_agg_registered_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 81349 entries, 2010-09-20 11:00:00 to 2019-12-31 23:00:00
Freq: H
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Registered  81349 non-null  int64
dtypes: int64(1)
memory usage: 1.2 MB


Select rows where Member type == 'Casual'  <br>
Then do groupby + agg to find total number of casual users per hour

In [92]:
df_agg_casual_hourly = df_all_years_hourly [df_all_years_hourly['Member type'] == 'Casual'].groupby(pd.Grouper(key = 'Start date', freq = 'H')).count().rename(columns={'Member type': 'Casual'})
df_agg_casual_hourly = df_agg_casual_hourly [['Casual']]


In [93]:
df_agg_casual_hourly.head()

,Casual
Start date,
2010-09-20 14:00:00,2
2010-09-20 15:00:00,2
2010-09-20 16:00:00,5
2010-09-20 17:00:00,2
2010-09-20 18:00:00,3


In [94]:
df_all_agg_hourly = pd.merge(df_agg_registered_hourly, df_agg_casual_hourly, how = 'outer', on = ['Start date'])

In [95]:
df_all_agg_hourly.head()
#df_all_agg_hourly[df_all_agg_hourly['Start date'] == '2011-01-01'].head(50)

,Registered,Casual
Start date,,
2010-09-20 11:00:00,2,NaN
2010-09-20 12:00:00,17,NaN
2010-09-20 13:00:00,11,NaN
2010-09-20 14:00:00,4,2.0
2010-09-20 15:00:00,10,2.0


In [96]:
df_all_agg_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 81349 entries, 2010-09-20 11:00:00 to 2019-12-31 23:00:00
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Registered  81349 non-null  int64  
 1   Casual      81346 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 1.9 MB


In [97]:
df_all_agg_hourly.reset_index(inplace = True)

In [98]:
df_all_agg_hourly['hour'] = df_all_agg_hourly['Start date'].dt.hour

In [99]:
df_all_agg_hourly['Start date'] = df_all_agg_hourly['Start date'].dt.date

In [100]:
df_all_agg_hourly.head()

,Start date,Registered,Casual,hour
0,2010-09-20,2,NaN,11
1,2010-09-20,17,NaN,12
2,2010-09-20,11,NaN,13
3,2010-09-20,4,2.0,14
4,2010-09-20,10,2.0,15


In [101]:
# But we know that NaN are all zeros. So:

df_all_agg_hourly['Registered'] = df_all_agg_hourly['Registered'].fillna(0)
df_all_agg_hourly['Casual'] = df_all_agg_hourly['Casual'].fillna(0)

In [102]:
df_all_agg_hourly.head()

,Start date,Registered,Casual,hour
0,2010-09-20,2,0.0,11
1,2010-09-20,17,0.0,12
2,2010-09-20,11,0.0,13
3,2010-09-20,4,2.0,14
4,2010-09-20,10,2.0,15


In [103]:
df_all_agg_hourly['Start date'] = pd.to_datetime(df_all_agg_hourly['Start date'])

In [104]:
df_all_agg_hourly['year'] = df_all_agg_hourly['Start date'].dt.year
df_all_agg_hourly['month'] = df_all_agg_hourly['Start date'].dt.month
df_all_agg_hourly['day'] = df_all_agg_hourly['Start date'].dt.day_name()
df_all_agg_hourly['day'] = df_all_agg_hourly['day'].apply(lambda x: x[0:3])

In [105]:
df_all_agg_hourly.head()

,Start date,Registered,Casual,hour,year,month,day
0,2010-09-20,2,0.0,11,2010,9,Mon
1,2010-09-20,17,0.0,12,2010,9,Mon
2,2010-09-20,11,0.0,13,2010,9,Mon
3,2010-09-20,4,2.0,14,2010,9,Mon
4,2010-09-20,10,2.0,15,2010,9,Mon


In [106]:
df_all_agg_hourly = df_all_agg_hourly.rename(columns = {'Start date': 'date',
                                    'Registered': 'registered',
                                    'Casual': 'casual'})

In [107]:
df_all_agg_daily.head()

,date,registered,casual,year,month,day
0,2010-09-20,178,34,2010,9,Mon
1,2010-09-21,215,109,2010,9,Tue
2,2010-09-22,260,117,2010,9,Wed
3,2010-09-23,249,124,2010,9,Thu
4,2010-09-24,206,156,2010,9,Fri


In [108]:
df_all_agg_daily ['year_month'] = df_all_agg_daily['date'].dt.to_period('M')
df_all_agg_hourly ['year_month'] = df_all_agg_hourly['date'].dt.to_period('M')

In [109]:
df_all_agg_daily.head()
df_all_agg_hourly.head()

,date,registered,casual,hour,year,month,day,year_month
0,2010-09-20,2,0.0,11,2010,9,Mon,2010-09
1,2010-09-20,17,0.0,12,2010,9,Mon,2010-09
2,2010-09-20,11,0.0,13,2010,9,Mon,2010-09
3,2010-09-20,4,2.0,14,2010,9,Mon,2010-09
4,2010-09-20,10,2.0,15,2010,9,Mon,2010-09


<h5> Export aggragated dataframes (daily + hourly) for the years 2010 – 2017 </h5>

In [110]:
# Export dataframes

df_all_agg_daily.to_csv('cleaned_data/df_all_agg_daily.csv', index = False)
df_all_agg_hourly.to_csv('cleaned_data/df_all_agg_hourly.csv', index = False)



In [111]:
df_all_years.head()

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,1012,2010-09-20 11:27:04,2010-09-20 11:43:56,31208,M St & New Jersey Ave SE,31108,4th & M St SW,W00742,Member
1,61,2010-09-20 11:41:22,2010-09-20 11:42:23,31209,1st & N St SE,31209,1st & N St SE,W00032,Member
2,2690,2010-09-20 12:05:37,2010-09-20 12:50:27,31600,5th & K St NW,31100,19th St & Pennsylvania Ave NW,W00993,Member
3,1406,2010-09-20 12:06:05,2010-09-20 12:29:32,31600,5th & K St NW,31602,Park Rd & Holmead Pl NW,W00344,Member
4,1413,2010-09-20 12:10:43,2010-09-20 12:34:17,31100,19th St & Pennsylvania Ave NW,31201,15th & P St NW,W00883,Member


In [112]:
df_all_years = df_all_years.rename(columns = {'Duration': 'duration',
                                'Start date': 'start_date',
                                'End date': 'end_date',
                                'Start station number': 'start_station_number',
                                'Start station': 'start_station',
                                'End station number': 'end_station_number',
                                'End station': 'end_station',
                                'Bike number': 'bike_number',
                                'Member type': 'member_type'
                                })

In [113]:
df_all_years.head(2)
df_all_years.tail(2)

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type
26058742,962,2019-12-31 23:59:38,2020-01-01 00:15:40,31236,37th & O St NW / Georgetown University,31214,17th & Corcoran St NW,W00534,Member
26058743,183,2019-12-31 23:59:55,2020-01-01 00:02:58,31227,13th St & New York Ave NW,31254,15th & K St NW,W22782,Member


In [114]:
df_all_years['registered'] = df_all_years['member_type'].apply(lambda x: 1 if x == 'Member' else 0)
df_all_years['casual'] = df_all_years['member_type'].apply(lambda x: 1 if x == 'Casual' else 0)

In [115]:
df_2011 = df_all_years[df_all_years['start_date'].dt.year == 2011]
df_2012 = df_all_years[df_all_years['start_date'].dt.year == 2012]
df_2013 = df_all_years[df_all_years['start_date'].dt.year == 2013]
df_2014 = df_all_years[df_all_years['start_date'].dt.year == 2014]
df_2015 = df_all_years[df_all_years['start_date'].dt.year == 2015]
df_2016 = df_all_years[df_all_years['start_date'].dt.year == 2016]
df_2017 = df_all_years[df_all_years['start_date'].dt.year == 2017]
df_2018 = df_all_years[df_all_years['start_date'].dt.year == 2018]
df_2019 = df_all_years[df_all_years['start_date'].dt.year == 2019]

In [116]:
df_2019.head(2)

,duration,start_date,end_date,start_station_number,start_station,end_station_number,end_station,bike_number,member_type,registered,casual
22660327,230,2019-01-01 00:04:48,2019-01-01 00:08:39,31203,14th & Rhode Island Ave NW,31200,Massachusetts Ave & Dupont Circle NW,E00141,Member,1,0
22660328,1549,2019-01-01 00:06:37,2019-01-01 00:32:27,31321,15th St & Constitution Ave NW,31114,18th St & Wyoming Ave NW,W24067,Casual,0,1


In [117]:
df_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3398417 entries, 22660327 to 26058743
Data columns (total 11 columns):
 #   Column                Dtype         
---  ------                -----         
 0   duration              int64         
 1   start_date            datetime64[ns]
 2   end_date              datetime64[ns]
 3   start_station_number  int64         
 4   start_station         object        
 5   end_station_number    int64         
 6   end_station           object        
 7   bike_number           object        
 8   member_type           object        
 9   registered            int64         
 10  casual                int64         
dtypes: datetime64[ns](2), int64(5), object(4)
memory usage: 311.1+ MB


<h5> Export "df_all_years" as csv file </h5>

In [118]:
df_all_years.to_csv('cleaned_data/df_all_years.csv', encoding = 'utf-8', index = False)

Run time: 2m 40 s

<h5> Export "df_2011" to "df_2020" as csv files </h5>

In [119]:
df_2011.to_csv('cleaned_data/df_2011.csv', encoding = 'utf-8', index = False)
df_2012.to_csv('cleaned_data/df_2012.csv', encoding = 'utf-8', index = False)
df_2013.to_csv('cleaned_data/df_2013.csv', encoding = 'utf-8', index = False)
df_2014.to_csv('cleaned_data/df_2014.csv', encoding = 'utf-8', index = False)
df_2015.to_csv('cleaned_data/df_2015.csv', encoding = 'utf-8', index = False)
df_2016.to_csv('cleaned_data/df_2016.csv', encoding = 'utf-8', index = False)
df_2017.to_csv('cleaned_data/df_2017.csv', encoding = 'utf-8', index = False)
df_2018.to_csv('cleaned_data/df_2018.csv', encoding = 'utf-8', index = False)
df_2019.to_csv('cleaned_data/df_2019.csv', encoding = 'utf-8', index = False)

Run time: 2 m 39s